# Concat Crdnt to TAAS Dataset

In [ ]:
taas = pd.read_csv('data/taas/ko/seoul.csv', encoding = 'cp949')
crdnt = pd.read_csv('data/taas/crdnt/seoul_crdnt.csv', encoding = 'cp949')

In [ ]:
print(f'taas: {len(taas)}')
print(f'crdnt: {len(crdnt)}')

taas: 108398
crdnt: 108398


In [ ]:
taas_df = pd.concat([taas, crdnt], axis=1)

In [ ]:
print(f'taas_df: {len(taas_df)}')
taas_df.head(1)

taas_df: 108398


,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,x_crdnt,y_crdnt
0,2019010100100001,2019년 1월 1일 00시,화요일,서울특별시 강서구 방화동,경상사고,0,0,1,0,차대사람 - 횡단중,...,승용,남,26세,상해없음,보행자,남,40세,경상,939508,1952236


# Filtering
합집합
- 가해운전자 차종: 이륜차
- 피해운전자 차종: 이륜차

In [ ]:
taas_df['가해운전자 차종'].unique()

array(['승용', '이륜', '화물', '기타불명', '특수', '승합', '원동기', '건설기계', '자전거',
       '개인형이동수단(PM)', '사륜오토바이(ATV)', '농기계'], dtype=object)

In [ ]:
taas_df = taas_df[(taas_df['가해운전자 차종'] == '이륜') | (taas_df['피해운전자 차종'] == '이륜')]
taas_df.reset_index(inplace=True)
taas_df.drop(columns = 'index', inplace=True)
print(f'taas_df: {len(taas_df)}')

taas_df: 25352


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
taas_df.to_csv('data/taas/before/seoul.csv', index = False, encoding = 'cp949')

# Convert location form

In [ ]:
!pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
taas_df = pd.read_csv('data/taas/before/seoul.csv', encoding = 'cp949')

In [ ]:
location = taas_df[['x_crdnt', 'y_crdnt']]
taas_df.drop(columns = ['x_crdnt', 'y_crdnt'], inplace=True)

In [ ]:
taas_df.head(1)

,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도
0,2019010100100002,2019년 1월 1일 00시,화요일,서울특별시 구로구 고척동,경상사고,0,0,1,0,차대차 - 추돌,...,맑음,단일로 - 기타,이륜,남,23세,상해없음,승용,남,71세,경상


In [ ]:
location.head(1)

,x_crdnt,y_crdnt
0,944074,1944702


In [ ]:
from pyproj import Proj, transform
 
epsg5179= Proj(init="epsg:5179")
wgs84=Proj(init='epsg:4326')

/usr/local/lib/python3.8/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.8/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
location['transformed'] = location.apply(lambda row : transform(epsg5179, wgs84, row['x_crdnt'], row['y_crdnt']), axis = 1)

location.head(1)

<ipython-input-189-c8d899f6864e>:1: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  location['transformed'] = location.apply(lambda row : transform(epsg5179, wgs84, row['x_crdnt'], row['y_crdnt']), axis = 1)
<ipython-input-189-c8d899f6864e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location['transformed'] = location.apply(lambda row : transform(epsg5179, wgs84, row['x_crdnt'], row['y_crdnt']), axis = 1)


,x_crdnt,y_crdnt,transformed
0,944074,1944702,"(126.86728619469758, 37.49988906509788)"


In [ ]:
taas_df['x'] = location['transformed'].apply(lambda x : x[0])
taas_df['y'] = location['transformed'].apply(lambda x : x[1])

taas_df.head(1)

,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,x,y
0,2019010100100002,2019년 1월 1일 00시,화요일,서울특별시 구로구 고척동,경상사고,0,0,1,0,차대차 - 추돌,...,이륜,남,23세,상해없음,승용,남,71세,경상,126.867286,37.499889


In [ ]:
print(f'taas_df: {len(taas_df)}')

taas_df: 25352


In [ ]:
taas_df.to_csv('data/taas/after/seoul.csv', index = False, encoding = 'cp949')

# Combine all dataframes

In [ ]:
seoul = pd.read_csv('data/taas/after/seoul.csv', encoding = 'cp949')
busan = pd.read_csv('data/taas/after/busan.csv', encoding = 'cp949')
daejeon = pd.read_csv('data/taas/after/daejeon.csv', encoding = 'cp949')
chungnam = pd.read_csv('data/taas/after/chungnam.csv', encoding = 'cp949')
sejong = pd.read_csv('data/taas/after/sejong.csv', encoding = 'cp949')
daegu = pd.read_csv('data/taas/after/daegu.csv', encoding = 'cp949')
incheon = pd.read_csv('data/taas/after/incheon.csv', encoding = 'cp949')
gwangju = pd.read_csv('data/taas/after/gwangju.csv', encoding = 'cp949')
ulsan = pd.read_csv('data/taas/after/ulsan.csv', encoding = 'cp949')

In [ ]:
print(f'seoul: {len(seoul)}')
print(f'busan: {len(busan)}')
print(f'daejeon: {len(daejeon)}')
print(f'chungnam: {len(chungnam)}')
print(f'sejong: {len(sejong)}')
print(f'daegu: {len(daegu)}')
print(f'incheon: {len(incheon)}')
print(f'gwangju: {len(gwangju)}')
print(f'ulsan: {len(ulsan)}')

seoul: 25352
busan: 8376
daejeon: 3467
chungnam: 4666
sejong: 409
daegu: 8881
incheon: 3951
gwangju: 3497
ulsan: 2816


In [ ]:
taas_only = pd.concat([seoul, busan, daejeon, chungnam, sejong, daegu, incheon, gwangju, ulsan])

In [ ]:
print(f'all location: {len(taas_only)}')

all location: 61415


In [ ]:
taas_only.to_csv('data/taas/taas_only.csv', index = False, encoding = 'cp949')